In [1]:
import os
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
from time import time
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
y_field = 'y:label'
data_file_name = '../data/kddcup.data_10_percent.gz'
# data_file_name = '../data/kddcup.data.gz'

In [3]:
features_types = ["duration: continuous.", "protocol_type: symbolic.", "service: symbolic.", "flag: symbolic.", "src_bytes: continuous.", "dst_bytes: continuous.", "land: symbolic.", "wrong_fragment: continuous.", "urgent: continuous.", "hot: continuous.", "num_failed_logins: continuous.", "logged_in: symbolic.", "num_compromised: continuous.", "root_shell: continuous.", "su_attempted: continuous.", "num_root: continuous.", "num_file_creations: continuous.", "num_shells: continuous.", "num_access_files: continuous.", "num_outbound_cmds: continuous.", "is_host_login: symbolic.", "is_guest_login: symbolic.", "count: continuous.", "srv_count: continuous.", "serror_rate: continuous.", "srv_serror_rate: continuous.", "rerror_rate: continuous.", "srv_rerror_rate: continuous.", "same_srv_rate: continuous.", "diff_srv_rate: continuous.", "srv_diff_host_rate: continuous.", "dst_host_count: continuous.", "dst_host_srv_count: continuous.", "dst_host_same_srv_rate: continuous.", "dst_host_diff_srv_rate: continuous.", "dst_host_same_src_port_rate: continuous.", "dst_host_srv_diff_host_rate: continuous.", "dst_host_serror_rate: continuous.", "dst_host_srv_serror_rate: continuous.", "dst_host_rerror_rate: continuous.", "dst_host_srv_rerror_rate: continuous.",'y:label']
features_0 = ["duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate","y"]
df = pd.read_csv(data_file_name, names=features_types)
print(df.shape)
df.head()

(494021, 42)


,duration: continuous.,protocol_type: symbolic.,service: symbolic.,flag: symbolic.,src_bytes: continuous.,dst_bytes: continuous.,land: symbolic.,wrong_fragment: continuous.,urgent: continuous.,hot: continuous.,...,dst_host_srv_count: continuous.,dst_host_same_srv_rate: continuous.,dst_host_diff_srv_rate: continuous.,dst_host_same_src_port_rate: continuous.,dst_host_srv_diff_host_rate: continuous.,dst_host_serror_rate: continuous.,dst_host_srv_serror_rate: continuous.,dst_host_rerror_rate: continuous.,dst_host_srv_rerror_rate: continuous.,y:label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [4]:
df['y_binary'] = 'benign'
df.loc[df[y_field]!='normal.','y_binary']='malware'
y_field = 'y_binary'

In [5]:
df['y_binary'].value_counts()

malware    396743
benign      97278
Name: y_binary, dtype: int64

In [6]:
n_classes = df[y_field].nunique()
n_classes

2

In [7]:
features_continuous = [x for x in features_types if 'continuous' in x]
features_categorical = ['protocol_type: symbolic.','service: symbolic.','flag: symbolic.']

print(len(features_continuous),len(features_categorical))

34 3


In [8]:
df = pd.get_dummies(df,columns=features_categorical)
df.shape

(494021, 120)

In [10]:
features_categorical_extended = [x for y in features_categorical for x in df.columns if x.split('_')[0]==y]
len(features_categorical_extended)

77

In [11]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold,KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,precision_score,classification_report,accuracy_score

In [12]:
# Class that identifies Column type
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, names=None):
        self.names = names
    def fit (self, X, y=None, **fit_params):
        return self
    def transform(self, X):
        return X[self.names]
    
class Logger(BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self,X, y=None):
        print("process done: {}".format(X.shape))
        return X

In [13]:
features = FeatureUnion(
    transformer_list=[
        ("numeric",Pipeline(
        [
            ('selector',ItemSelector(names=features_continuous)),
            ('scaler',MinMaxScaler()),
            ('log',Logger()),
        ])
        ),
        
        
         ("categorical",Pipeline(
        [
            ('selector',ItemSelector(names=features_categorical_extended)),
            ('log',Logger()),
        ])
        ),
#         ('log',Logger()),
    ]
)

In [14]:
cls = SVC()
cls = RandomForestClassifier(n_jobs=10, max_depth=5, n_estimators=10, max_features=1)
cls = LogisticRegression()
simple_pipeline = Pipeline(
[
    ('features',features),
    ('cls',cls)
],
)

In [ ]:
%%notify
t = time()
cv = StratifiedKFold(n_splits=2, shuffle=True)
# cv = StratifiedShuffleSplit(n_splits=3,test_size=0.1)
cms = np.ndarray((cv.n_splits, n_classes * n_classes), int)
for i, (train, test) in zip(range(cv.n_splits),cv.split(df,df[y_field])):
    simple_pipeline.fit(df.iloc[train],df.iloc[train][y_field])
    y_pred = simple_pipeline.predict(df.iloc[test])
    cm = confusion_matrix(df.iloc[test][y_field], y_pred)
#     print(cm)
    print("p_micro:{},p_macro:{}".format(precision_score(df.iloc[test][y_field],y_pred,average='micro'),
                                        precision_score(df.iloc[test][y_field],y_pred,average='macro')))
    print(classification_report(df.iloc[test][y_field],y_pred))
    cms[i] = cm.flatten()
avg_cms = cms.mean(axis=0).reshape((n_classes, n_classes)).astype(int)
print('average confusion matrix:\n{}'.format(avg_cms))
print('estimation finished. Elapsed: {}'.format(time() - t))

## grid search

In [15]:
from sklearn.model_selection  import GridSearchCV,RandomizedSearchCV

In [16]:
simple_pipeline.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('numeric', Pipeline(memory=None,
       steps=[('selector', ItemSelector(names=['duration: continuous.', 'src_bytes: continuous.', 'dst_bytes: continuous.', 'wrong_fragment: continuous.', 'urgent: continuous.', 'hot: continuous.', 'num_failed_logins: continuous.', 'num_compromised: ...lic._S2', 'flag: symbolic._S3', 'flag: symbolic._SF', 'flag: symbolic._SH'])), ('log', Logger())]))],
         transformer_weights=None)),
 ('cls',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [18]:
n_estimators = list(range(200,1000,100))

max_features = ['auto', 'sqrt']
max_depth =  list(range(10,110,11))
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'cls__n_estimators': n_estimators,
#                'cls__max_features': max_features,
               'cls__max_depth': max_depth,
               'cls__min_samples_split': min_samples_split,
               'cls__min_samples_leaf': min_samples_leaf,
               'cls__bootstrap': bootstrap}
print(random_grid)

{'cls__n_estimators': [200, 300, 400, 500, 600, 700, 800, 900], 'cls__max_depth': [10, 21, 32, 43, 54, 65, 76, 87, 98, 109, None], 'cls__min_samples_split': [2, 5, 10], 'cls__min_samples_leaf': [1, 2, 4], 'cls__bootstrap': [True, False]}


In [20]:
rf_random = RandomizedSearchCV(simple_pipeline,
                               param_distributions=random_grid,
                               scoring='accuracy',
                               n_iter=10,
                               cv=3,
                               verbose=1,
                               random_state=17,
                               n_jobs=4)

In [ ]:
%%notify
t = df.sample(1000)
rf_random.fit(t[features_continuous], t[y_field])

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
2+2

In [ ]:
simple_pipeline.fit(df,df[y_field])

# Estimation

In [ ]:
df_test = pd.read_csv('../data/corrected.gz',names=features_types)
print(df_test.shape)
df_test.head()

In [ ]:
df_test['y_binary'] = 'benign'
df_test.loc[df_test['y:label']!='normal.','y_binary']='malware'
y_field = 'y_binary'
df_test = pd.get_dummies(df_test,columns=features_categorical)
df_test['y_binary'].value_counts()

TEST FEATURES != LEARN FEATURES

In [ ]:
df_test.columns

In [ ]:
df_test[features_categorical_extended]

In [ ]:
df_test['y_pred'] = simple_pipeline.predict(df_test)
print(classification_report(df_test[y_field],df_test['y_pred']))
print(f"acc:{accuracy_score(df_test[y_field],df_test['y_pred'])}")
print(confusion_matrix(df_test[y_field],df_test['y_pred']))

In [ ]:
accuracy_score(df_test[y_field],df_test['y_pred'])

# Plan
* other features - one hot encoding
* add grid search
* NN